## Libraries - Packages

In [1]:
import json
import pandas as pd
import networkx as nx
import os
from tqdm import tqdm
from os.path import join
from data_handler import FeatureEngineering
import gc

import numpy as np
import matplotlib.pyplot as plt

from torch.nn import Linear
from torch.nn import ReLU
from torch.nn.init import kaiming_uniform_
import torchmetrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric_temporal.nn.recurrent import A3TGCN2
import random
from torch.utils.data import TensorDataset, DataLoader

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

PyTorch version: 2.6.0
PyTorch Geometric version: 2.3.1


In [2]:
def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For CUDA devices
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds()


## Global Variables

In [3]:
input_path = join("io", "input")
output_path = join("io", "output")
experiments_path = join("io", "experiments")
graph_structured_np = join(output_path, "graph_structured_np")
metrics_path = join(experiments_path, "metrics")
plots_path = join(experiments_path, "plots")
gnn_model_path = join("io", "gnn_model")

core_features = ['node_id', 'ETA_curr']

calendar_features = ['sin_hour','cos_hour', 'sin_dayofweek', 'cos_dayofweek', 'sin_month', 'cos_month','sin_dayofmonth', 'cos_dayofmonth', 
                     'sin_weekofyear', 'cos_weekofyear', 'sin_quarter_hour', 'cos_quarter_hour']

rolling_avg_features = ['rolling_avg_4h', 'rolling_avg_12h', 'rolling_avg_68h', 'rolling_avg_476h', 'rolling_avg_20240h']

lag_features = ['lag1h', 'lag4h', 'lag476h', 'lag20240h']

feature_cols = core_features + calendar_features + rolling_avg_features + lag_features

edges_df = pd.read_csv(join(output_path, "interm_network_edges.csv"), encoding='utf-8', sep=',')


# Graph Representation Process

## Create Graph Edges

In [4]:
def calculate_edges_info(df, node_id_to_index, fe):
    edge_index = []
    edge_weights = []
    
    # Iterating with index to ensure you use the correct node references
    for i in tqdm(range(len(df))):
        node_i = df.iloc[i]
        node_id_i = node_id_to_index[node_i['edge_id']]
        for j in range(i + 1, len(df)):
            node_j = df.iloc[j]
            node_id_j = node_id_to_index[node_j['edge_id']]
    
            # Check if any of the start or end node ids match
            if any(node_i[['start_node_id', 'end_node_id']].isin(node_j[['start_node_id', 'end_node_id']])):
                edge_index.append([node_id_i, node_id_j])
                weight = fe.haversine(node_i['start_lat'], node_i['start_long'], node_j['start_lat'], node_j['start_long'])
                edge_weights.append(weight)
    
    # Convert lists to numpy arrays for use in data structures
    edge_index = np.array(edge_index).T  # Shape: [2, num_edges]
    edge_weights = np.array(edge_weights)

    # Normalize edge weights from 0 to 1
    edge_weights = (edge_weights - edge_weights.min()) / (edge_weights.max() - edge_weights.min())
    
    return edge_index, edge_weights
    

## Convert Tabular Data to Numpy Based Graph Structure

In [5]:
def load_and_concat_arrays(save_dir, years):
    # You presumably have the same node_id_to_index across all years
    # The final shape for node dimension and features dimension must match
    features_list = []
    targets_list = []
    timestamps_list = []

    for year in years:
        fpath_features = os.path.join(save_dir, f"features_year_{year}.npy")
        fpath_targets = os.path.join(save_dir, f"targets_year_{year}.npy")
        fpath_timestamps = os.path.join(save_dir, f"timestamps_year_{year}.npy")

        feat_local = np.load(fpath_features)
        targ_local = np.load(fpath_targets)
        ts_local   = np.load(fpath_timestamps)

        features_list.append(feat_local)
        targets_list.append(targ_local)
        timestamps_list.append(ts_local)

    # Concatenate along time dimension
    features_all = np.concatenate(features_list, axis=0)
    targets_all  = np.concatenate(targets_list, axis=0)
    timestamps_all = np.concatenate(timestamps_list, axis=0)

    return features_all, targets_all, timestamps_all


In [6]:
def convert_df_to_numpy(df, feature_cols, poi_cols, save_dir=graph_structured_np, dtype=np.float32):
    """
    Split the DataFrame by year, create partial arrays for each year,
    then immediately save each partial array to disk, freeing memory.

    Parameters
    ----------
    df : pd.DataFrame
        Contains columns: 'node_id', 'timestamp', 'target', ...
        'timestamp' should be datetime or convertible to datetime.
    feature_cols : list
        Features besides 'node_id', 'timestamp', 'target'.
    poi_cols : list
        Extra columns for points of interest distances, appended to feature_cols.
    save_dir : str
        Directory path in which to save arrays for each year (create if needed).
    dtype : numpy.dtype
        e.g. np.float32 or np.float16 to reduce memory usage.

    Returns
    -------
    node_id_to_index : dict
        The global mapping of node_id -> node index.
    sorted_years : list
        Sorted list of all years processed, in case you want them for reference.
    """

    # Ensure output directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    df = df.copy()
    if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Build a global node->index mapping across all data
    unique_nodes = df['node_id'].unique()
    node_id_to_index = {nid: i for i, nid in enumerate(unique_nodes)}
    num_nodes = len(unique_nodes)

    # Combine feature + poi columns, excluding 'node_id'
    feature_cols = [col for col in feature_cols if col != 'node_id']
    all_feature_cols = feature_cols + poi_cols

    # Add 'year' col to split by year
    df['year'] = df['timestamp'].dt.year
    sorted_years = sorted(df['year'].unique())

    for year in sorted_years:
        print(f"\nProcessing year={year}...")

        # Filter DataFrame for this year
        df_year = df[df['year'] == year].copy()
        # Sort by timestamp for stable ordering
        df_year.sort_values(by='timestamp', inplace=True)

        # Unique timestamps for this year
        local_timestamps = df_year['timestamp'].unique()
        local_ts_to_idx = {ts: i for i, ts in enumerate(local_timestamps)}
        num_timesteps = len(local_timestamps)

        # Prepare partial arrays
        num_features = len(all_feature_cols)
        features_local = np.zeros((num_timesteps, num_nodes, num_features), dtype=dtype)
        targets_local = np.zeros((num_timesteps, num_nodes, 1), dtype=dtype)

        # Fill arrays
        for _, row in tqdm(df_year.iterrows(), total=df_year.shape[0], desc=f"Year={year}"):
            node_idx = node_id_to_index[row['node_id']]
            time_idx = local_ts_to_idx[row['timestamp']]

            features_local[time_idx, node_idx, :] = row[all_feature_cols].values
            targets_local[time_idx, node_idx, 0] = row['target']

        # Save to disk
        # e.g. np.save for each, or np.savez to store them together
        np.save(os.path.join(save_dir, f"features_year_{year}.npy"), features_local)
        np.save(os.path.join(save_dir, f"targets_year_{year}.npy"), targets_local)
        np.save(os.path.join(save_dir, f"timestamps_year_{year}.npy"), local_timestamps)

        # Free memory
        del features_local, targets_local, df_year
        gc.collect()

    # Optionally drop the added 'year' column
    df.drop(columns=['year'], inplace=True, errors='ignore')

    features_all, targets_all, timestamps_all = load_and_concat_arrays(save_dir, sorted_years)

    print("\nFinished saving partial arrays year by year.")
    return features_all, targets_all, node_id_to_index, timestamps_all


In [7]:
def prepare_future_targets(targets, num_periods):
    # Assuming targets is of shape (2039, 225, 1)
    num_timesteps, num_nodes, _ = targets.shape
    # Check if there's enough data to form the future targets
    if num_timesteps < num_periods:
        raise ValueError("Not enough timesteps to form future targets.")
    
    # Initialize the future targets array
    # Shape will be (1972, 225, 68) since we need 68 future data points for each of the starting 1972 points
    future_targets = np.zeros((num_timesteps - num_periods + 1, num_nodes, num_periods))

    # Populate the future targets array
    for i in range(future_targets.shape[0]):
        for n in range(num_nodes):
            future_targets[i, n, :] = targets[i:i + num_periods, n, 0]  # reshape or squeeze as necessary

    return future_targets
    

## Create Torch Data Loaders

In [8]:
def static_graph_data_loader(features, targets, edge_index, edge_weights, set_nanme, batch_size, num_periods):
    # Convert features to a PyTorch tensor
    features_tensor = torch.from_numpy(features).type(torch.FloatTensor)  # Initial Dims = (2039, 225, 38)

    # Create a time window for each feature at each node
    # Unfold along the first dimension (time) to create windows
    features_tensor = features_tensor.unfold(0, num_periods, 1)  # Now dims = (1972, 225, 38, 68)

    # No need to permute since it is already in the desired order: (timesteps, nodes, features, periods)
    
    # Prepare targets accordingly (assuming prepare_future_targets function adjusts them appropriately)
    prepared_targets = prepare_future_targets(targets, num_periods)
    targets_tensor = torch.from_numpy(prepared_targets).type(torch.FloatTensor)

    # Check dimensions
    print(f"{set_nanme} Features tensor shape: {features_tensor.shape}")
    print(f"{set_nanme} Targets tensor shape: {targets_tensor.shape}")

    # Convert edge data to tensors
    edge_index_tensor = torch.from_numpy(edge_index).type(torch.LongTensor) # Dims = (2, 292)
    edge_weights_tensor = torch.from_numpy(edge_weights).type(torch.FloatTensor) # Dims = (292)

    # Ensure that both tensors are aligned in the first dimension
    assert features_tensor.shape[0] == targets_tensor.shape[0], "Feature and target tensor size mismatch"

    # Create dataset and loader
    dataset = TensorDataset(features_tensor, targets_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    return dataloader, edge_index_tensor, edge_weights_tensor


In [9]:
def create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods, final_training=False):

    if use_validation:
        train_name = 'train'
        eval_name = 'validation'
    else:
        if final_training:
            train_name = 'train_final'
            eval_name = 'test_final'
        else:
            train_name = 'train_full'
            eval_name = 'test'
    
    if pre_calculated:
        train_features = np.load(join(output_path, 'fct_' + train_name + '_features.npy'))
        train_targets = np.load(join(output_path, 'fct_' + train_name + '_target.npy'))
        eval_features = np.load(join(output_path, 'fct_' + eval_name + '_features.npy'))
        eval_targets = np.load(join(output_path, 'fct_' + eval_name + '_target.npy'))
        edge_index = np.load(join(output_path, 'fct_edge_index.npy'))
        edge_weights = np.load(join(output_path, 'fct_edge_weights.npy')) 
    else:
        fe = FeatureEngineering (use_validation, feature_cols, training_end_date, validation_end_date, test_end_date)
        train_df, eval_df, poi_columns = fe.get_datasets()     
        print('Convert train set to Numpy arrays')
        train_features, train_targets, node_id_to_index, timestamp_to_index = convert_df_to_numpy(train_df, feature_cols, poi_columns)
        del timestamp_to_index
        print('Convert evaluation set to Numpy arrays')
        eval_features, eval_targets, test_node_id_to_index, test_timestamp_to_index = convert_df_to_numpy(eval_df, feature_cols, poi_columns)

        print('Create edge info')
        edges_df = pd.read_csv(join(output_path, "interm_network_edges.csv"), encoding='utf-8', sep=',')
        edge_index, edge_weights = calculate_edges_info(edges_df, node_id_to_index, fe)

        print('Save Numpy arrays')
        if save:
            np.save(join(output_path, 'fct_' + train_name + '_features.npy'), train_features)
            np.save(join(output_path, 'fct_' + train_name + '_target.npy'), train_targets)
            np.save(join(output_path, 'fct_' + eval_name + '_features.npy'), eval_features)
            np.save(join(output_path, 'fct_' + eval_name + '_target.npy'), eval_targets)
            np.save(join(output_path, 'fct_edge_index.npy'), edge_index)
            np.save(join(output_path, 'fct_edge_weights.npy'), edge_weights)
            if final_training:
                np.save(join(output_path, 'fct_predicted_timestamps.npy'), test_timestamp_to_index)
                save_dict(output_path, 'fct_predicted_nodes.npy', test_node_id_to_index)

    num_features = train_features.shape[2]
    print('Train Features Shape:', train_features.shape)
    print('Train Targets Shape:', train_targets.shape)
    print('Evaluation Features Shape:', eval_features.shape)
    print('Evaluation Targets Shape:', eval_targets.shape)
    print("Edge Index shape:", edge_index.shape)
    print("Edge Weights shape:", edge_weights.shape)

    train_dataloader, edge_index_tensor, edge_weights_tensor = static_graph_data_loader(train_features, train_targets, edge_index, edge_weights, 'Training', batch_size, periods)
    eval_dataloader, edge_index_tensor, edge_weights_tensor = static_graph_data_loader(eval_features, eval_targets, edge_index, edge_weights, 'Evaluation', batch_size, periods)

    return train_dataloader, eval_dataloader, edge_index_tensor, edge_weights_tensor, num_features
    

## Graph Neural Networks Architecture

In [10]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size, hidden_layers):
        super(TemporalGNN, self).__init__()
        
        # Initial temporal graph convolution layer
        self.tgnn = A3TGCN2(in_channels=node_features, out_channels=hidden_layers[0], periods=periods, batch_size=batch_size)

        # Dynamically create hidden layers based on the 'hidden_layers' list
        self.hidden_layers = torch.nn.ModuleList()
        for i in range(1, len(hidden_layers)):
            self.hidden_layers.append(Linear(hidden_layers[i - 1], hidden_layers[i]))
            self.hidden_layers.append(ReLU())

        # Output layers for mu and log variance
        self.fc_mu = Linear(hidden_layers[-1], periods)
        self.fc_logvar = Linear(hidden_layers[-1], periods)

    def forward(self, x, edge_index, edge_weight):
        h = self.tgnn(x, edge_index)
        h = F.relu(h)

        # Pass through dynamically created hidden layers
        for layer in self.hidden_layers:
            h = layer(h)
        
        mu = self.fc_mu(h)
        log_var = self.fc_logvar(h)
        return mu, log_var


TemporalGNN(node_features=38, periods=1, batch_size=64, hidden_layers=[128,64,32])


TemporalGNN(
  (tgnn): A3TGCN2(
    (_base_tgcn): TGCN2(
      (conv_z): GCNConv(38, 128)
      (linear_z): Linear(in_features=256, out_features=128, bias=True)
      (conv_r): GCNConv(38, 128)
      (linear_r): Linear(in_features=256, out_features=128, bias=True)
      (conv_h): GCNConv(38, 128)
      (linear_h): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (hidden_layers): ModuleList(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (fc_mu): Linear(in_features=32, out_features=1, bias=True)
  (fc_logvar): Linear(in_features=32, out_features=1, bias=True)
)

## Probabilistic Forecasting Approach

In [11]:
def gaussian_nll(mu, logvar, target):
    return (0.5 * torch.exp(-logvar) * (target - mu) ** 2 + 0.5 * logvar).mean()


## Training & Evaluation

In [12]:
def evaluate_model(model, eval_loader, loss_fn, edge_index_tensor, edge_weights_tensor):
    epsilon = 1e-8
    model.eval()
    with torch.no_grad():
        loss_list, mae_list, mape_list, r2_list, rmse_list = [], [], [], [], []

        for encoder_inputs, labels in eval_loader:
            # Forward pass through the model
            mu, log_var = model(encoder_inputs, edge_index_tensor, edge_weights_tensor)
            
            # Calculate loss
            loss = gaussian_nll(mu, log_var, labels)
            loss_list.append(loss.item())

            predictions = mu
            # Calculate MAE
            mae = torch.mean(torch.abs(predictions - labels))
            mae_list.append(mae.item())

            # Calculate MAPE
            mape = torch.mean(torch.abs(predictions - labels) / (torch.abs(labels) + epsilon))
            mape_list.append(mape.detach().numpy().item())

            # Calculate R2 Score
            r2 = torchmetrics.functional.r2_score(predictions.view(-1), labels.view(-1))
            r2_list.append(r2.item())

            # Calculate RMSE
            rmse = torch.sqrt(torch.mean(torch.pow(predictions - labels, 2)))
            rmse_list.append(rmse.item())

        # Aggregate metrics
        avg_loss = sum(loss_list) / len(loss_list)
        avg_mae = sum(mae_list) / len(mae_list)
        avg_mape = sum(mape_list) / len(mape_list)
        avg_r2 = sum(r2_list) / len(r2_list)
        avg_rmse = sum(rmse_list) / len(rmse_list)
        avg_rmse = sum(rmse_list) / len(rmse_list)

    return avg_loss, avg_mae, avg_mape, avg_r2, avg_rmse
    

In [13]:
def model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index_tensor, edge_weights_tensor, validation_mode):

    model=TemporalGNN(node_features=num_features, periods=periods, batch_size=batch_size, hidden_layers=hidden_layers)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    loss_fn = torch.nn.MSELoss()
    model.train()
    epsilon = 1e-8
    
    train_loss_ls, train_mae_ls, train_r2_ls, train_rmse_ls, train_mape_ls = [], [], [], [], []
    eval_loss_ls, eval_mae_ls, eval_r2_ls, eval_rmse_ls, eval_mape_ls = [], [], [], [], []
    
    for epoch in range(0,epochs):
        loss_list, mae_list ,r2_list, rmse_list, mape_list = [], [], [], [], []
        step = 0
        for encoder_inputs, labels in tqdm(train_dataloader): 
            mu, log_var = model(encoder_inputs, edge_index_tensor, edge_weights_tensor)       # Get model predictions
            loss = gaussian_nll(mu, log_var, labels)
            loss.backward()
            loss_list.append(loss.item())
            optimizer.step()
            optimizer.zero_grad()
            step = step + 1
            
            y_pred = mu
            y_true = labels
            
            mae = torch.mean(torch.abs(y_pred - y_true))
            mae_list.append(mae.detach().numpy().item())

            mape = torch.mean(torch.abs(y_pred - y_true) / (torch.abs(y_true) + epsilon))
            mape_list.append(mape.detach().numpy().item())
            
            r2 = torchmetrics.functional.r2_score(y_pred.view(-1), y_true.view(-1))
            r2_list.append(r2.detach().numpy().item())
            
            rmse = torch.sqrt(torch.mean(torch.pow(y_pred - y_true, 2)))
            rmse_list.append(rmse.detach().numpy().item())
    
        train_loss = sum(loss_list) / len(loss_list)
        train_mae = sum(mae_list) / len(mae_list)
        train_mape = sum(mape_list) / len(mape_list)
        train_r2 = sum(r2_list) / len(r2_list)
        train_rmse = sum(rmse_list) / len(rmse_list)
        
        print("Epoch {}, Train || NLL: {:.7f}, MAE: {:.7f}, MAPE: {:.7f}, R2: {:.7f}, RMSE: {:.7f}".format(epoch+1,train_loss,train_mae,train_mape,train_r2,train_rmse))
        train_loss_ls.append(train_loss)
        train_mae_ls.append(train_mae)
        train_mape_ls.append(train_mape)
        train_r2_ls.append(train_r2)
        train_rmse_ls.append(train_rmse)

        if validation_mode:
            eval_loss,eval_mae,eval_mape,eval_r2,eval_rmse = evaluate_model(model, eval_dataloader, loss_fn, edge_index_tensor, edge_weights_tensor)
            print("Epoch {}, Evaluation || NLL: {:.7f}, MAE: {:.7f}, MAPE: {:.7f}, R2: {:.7f}, RMSE: {:.7f}".format(epoch+1,eval_loss,eval_mae,eval_mape,eval_r2,eval_rmse))
            eval_loss_ls.append(eval_loss)
            eval_mae_ls.append(eval_mae)
            eval_mape_ls.append(eval_mape)
            eval_r2_ls.append(eval_r2)
            eval_rmse_ls.append(eval_rmse)

    metrics = {'train_nll_loss_ls': train_loss_ls,'train_mae_ls': train_mae_ls,'train_mape_ls': train_mape_ls,'train_r2_ls': train_r2_ls,
               'train_rmse_ls': train_rmse_ls,'eval_nll_loss_ls': eval_loss_ls,'eval_mae_ls': eval_mae_ls,'eval_mape_ls': eval_mape_ls,
               'eval_r2_ls': eval_r2_ls,'eval_rmse_ls': eval_rmse_ls}
    return model, metrics


In [14]:
def generate_forecasts(model, eval_dataloader, edge_index_tensor, edge_weights_tensor, node_id_mapping, timestamps):
    """
    Generates forecasts for the given model and dataloader, associating predictions with node IDs and timestamps.
    
    Parameters:
        model (torch.nn.Module): Trained model ready for predictions.
        eval_dataloader (DataLoader): DataLoader containing the evaluation data.
        edge_index_tensor (Tensor): Edge indices for the graph.
        edge_weights_tensor (Tensor): Edge weights for the graph.
        node_id_mapping (dict): Mapping of node indices to node IDs.
        timestamps (np.array): Array of timestamps corresponding to the evaluations.
    
    Returns:
        pd.DataFrame: DataFrame containing node IDs, timestamps, mu, and log_var.
    """
    model.eval()
    forecasts = []
    index_to_node_id = {v: k for k, v in node_id_mapping.items()}
    with torch.no_grad():
        for batch_idx, (encoder_inputs, _) in enumerate(eval_dataloader):
            mu, log_var = model(encoder_inputs, edge_index_tensor, edge_weights_tensor)

            # Iterate over each timestamp and node in the batch
            for idx in range(mu.size(0)):
                for node_idx in range(mu.size(1)):
                    node_id = index_to_node_id[node_idx]  # Correctly map index back to node ID using the inverted dictionary
                    timestamp = timestamps[batch_idx * eval_dataloader.batch_size + idx]
                    forecasts.append({
                        "node_id": node_id,
                        "timestamp": timestamp,
                        "mean": mu[idx, node_idx].item(),
                        "log_var": log_var[idx, node_idx].item()
                    })

    return pd.DataFrame(forecasts)

## Save Results

In [15]:
def save_dict(path, filename, data):
    with open(path + '/' + filename + '.json', 'w') as f:
        json.dump(data, f)
        

In [16]:
def load_dict(path, filename):
    with open(path + '/' + filename + '.json', 'r') as f:
        data_loaded = json.load(f)
    return data_loaded


## Execution Varibales

In [ ]:
training_end_date = '2023-01-01'
validation_end_date = '2024-01-01'
test_end_date = '2025-01-10'
use_validation = True
pre_calculated = False
save = True
periods = 1
batch_size = 32

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)


# Experiments

In [ ]:
training_end_date = '2023-01-01'
validation_end_date = '2024-01-01'
test_end_date = '2025-01-10'
use_validation = True
pre_calculated = True
save = False
periods = 1


## Experiments With Network Parameters

In [ ]:
hidden_layers_ls = [ [64], [128,64], [128,64,32], [256,128,64,32], [512,256,128,64,32] ]
model_name_ls = ['A3TGCN2', 'A3TGCN2_1hid', 'A3TGCN2_2hid', 'A3TGCN2_3hid', 'A3TGCN2_4hid']
batch_size = 32
epochs = 60
lr = 0.001

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)

for i in range (0,len(model_name_ls)):
    hidden_layers = hidden_layers_ls[i]
    print('Experiment with Model = ', model_name_ls[i])
    experiment_name = 'experiment_model_' + str(model_name_ls[i])
    model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
    save_dict(metrics_path, experiment_name, metrics)
    print('\n\n')
    

## Experiments With Epochs

In [ ]:
hidden_layers = [256,128,64,32]
epochs = 120
batch_size = 32
lr = 0.001

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)
model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
experiment_name = 'experiment_epochs_' + str(epochs)
save_dict(metrics_path, experiment_name, metrics)


## Experiments with Batch Size

In [ ]:
hidden_layers = [256,128,64,32]
batch_size_ls = [8, 16, 32, 64]

epochs = 60
lr = 0.001

for batch_size in batch_size_ls:
    print('Experiment with batch size = ', batch_size)
    experiment_name = 'experiment_batch_size_' + str(batch_size)
    train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)
    model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
    save_dict(metrics_path, experiment_name, metrics)
    print('\n\n')


## Experiments with Learning Rate

In [ ]:
hidden_layers = [256,128,64,32]
batch_size_ls = 32
epochs = 60
learning_rate_ls = [0.01, 0.001, 0.0001]

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)

for lr in learning_rate_ls:
    print('Experiment with Learning Rate = ', lr)
    experiment_name = 'experiment_learning_rate_' + str(lr)
    model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
    save_dict(metrics_path, experiment_name, metrics)
    print('\n\n')
    

## Ablation Study

In [ ]:
hidden_layers = [256,128,64,32]
pre_calculated = False
save = False
feature_names = ['core_features', 'calendar_features', 'rolling_avg_features', 'lag_features']

batch_size = 32
epochs = 60
lr = 0.001

for feature in feature_names:

    if feature == 'core_features':
        feature_cols = core_features
    elif feature == 'calendar_features':
        feature_cols = core_features + calendar_features
    elif feature == 'rolling_avg_features':
        feature_cols = core_features + calendar_features + rolling_avg_features
    elif feature == 'lag_features':
        feature_cols = core_features + calendar_features + rolling_avg_features + lag_features

    print('Experiment with features = ', feature)
    experiment_name = 'experiment_features_' + str(feature)
    train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)
    model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
    save_dict(metrics_path, experiment_name, metrics)
    print('\n\n')
    

## Calendar Features Analysis

In [ ]:
hidden_layers = [256,128,64,32]
pre_calculated = False
save = False
feature_names = ['hour_quarter', 'dayofweek', 'month', 'dayofmonth', 'weekofyear','all']

batch_size = 32
epochs = 60
lr = 0.001

for feature in feature_names:

    if feature == 'hour_quarter':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour']
    elif feature == 'dayofweek':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour','sin_dayofweek','cos_dayofweek']
    elif feature == 'month':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour','sin_dayofweek','cos_dayofweek','sin_month','cos_month']
    elif feature == 'dayofmonth':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour','sin_dayofweek','cos_dayofweek','sin_month','cos_month','sin_dayofmonth','cos_dayofmonth']
    elif feature == 'weekofyear':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour','sin_dayofweek','cos_dayofweek','sin_month','cos_month','sin_dayofmonth','cos_dayofmonth','sin_weekofyear','cos_weekofyear']
    elif feature == 'all':
        feature_cols = core_features + ['sin_hour','cos_hour','sin_quarter_hour','cos_quarter_hour','sin_dayofweek','cos_dayofweek','sin_month','cos_month','sin_dayofmonth','cos_dayofmonth','sin_weekofyear','cos_weekofyear','sin_year','cos_year']

    print('Experiment with calendar features = ', feature)
    experiment_name = 'experiment_calendar_features_' + str(feature)
    train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)
    model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
    save_dict(metrics_path, experiment_name, metrics)
    print('\n\n')
    

## Experiments with Testset

In [ ]:
training_end_date = '2023-01-01'
validation_end_date = '2024-01-01'
test_end_date = '2025-01-10'

use_validation = False
pre_calculated = True
save = False
periods = 1


In [ ]:
hidden_layers = [256,128,64,32]
epochs = 80
batch_size = 32
lr = 0.001

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods)
model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, True)
experiment_name = 'final_model_test_metrics'
save_dict(metrics_path, experiment_name, metrics)


## Full Train

In [ ]:
training_end_date = '2023-01-01'
validation_end_date = '2025-01-09'
test_end_date = '2025-01-10'

final_training = True
use_validation = False
pre_calculated = True
save = False
periods = 1

hidden_layers = [256,128,64,32]
epochs = 20
batch_size = 32
lr = 0.001

train_dataloader, eval_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, test_end_date, output_path, pre_calculated, save, feature_cols, batch_size, periods, final_training)
model, metrics = model_training(train_dataloader, eval_dataloader, epochs, num_features, periods, batch_size, hidden_layers, lr, edge_index, edge_weights, False)
torch.save(model.state_dict(), join(gnn_model_path, "TGCN_model.pt"))

## Generate Predictions

In [21]:
training_end_date = '2023-01-01'
validation_end_date = '2025-01-09'
pred_date = '2025-01-10'

final_training = True
use_validation = False
pre_calculated = True
save = False

hidden_layers = [256,128,64,32]
batch_size = 32
periods = 1

train_dataloader, pred_dataloader, edge_index, edge_weights, num_features = create_graph_data(use_validation, training_end_date, validation_end_date, pred_date, output_path, pre_calculated, save, feature_cols, batch_size, periods, final_training)
timestamps = np.load(join(output_path, "fct_predicted_timestamps.npy"), allow_pickle=True)
node_id_mapping = load_dict(output_path, "fct_predicted_nodes")

pre_trained_model = TemporalGNN(node_features=num_features, periods=periods, batch_size=batch_size, hidden_layers=hidden_layers)
pre_trained_model.load_state_dict(torch.load(join(gnn_model_path, "TGCN_model.pt")))

forecast_df = generate_forecasts(pre_trained_model, pred_dataloader, edge_index, edge_weights, node_id_mapping, timestamps)
forecast_df.to_csv(join(output_path, "interm_avg_speed_forecasts.csv"), sep=',', encoding='utf-8', index=False)

Train Features Shape: (75072, 225, 41)
Train Targets Shape: (75072, 225, 1)
Evaluation Features Shape: (68, 225, 41)
Evaluation Targets Shape: (68, 225, 1)
Edge Index shape: (2, 292)
Edge Weights shape: (292,)
Training Features tensor shape: torch.Size([75072, 225, 41, 1])
Training Targets tensor shape: torch.Size([75072, 225, 1])
Evaluation Features tensor shape: torch.Size([68, 225, 41, 1])
Evaluation Targets tensor shape: torch.Size([68, 225, 1])
